In [ ]:
# Compute global stats for intensity normalization

import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

PATH = "/path/to/AbdomenCT-1K/"
ID_FILE = "/path/to/train_ids.txt"
OUT_FILE = "/path/to/intensity_stats.csv"

In [ ]:
# Foreground voxels
i_vals_fg = []
ids = [l.rstrip('\n') for l in open(ID_FILE, 'r').readlines()]
for i in tqdm(ids):
    img = nib.load(os.path.join(PATH, i, "registered_img.nii.gz")).get_fdata()
    if not np.isfinite(img).all():
        raise ValueError(f"Image {i} not finite.")
    label = nib.load(os.path.join(PATH, i, "registered_label.nii.gz")).get_fdata()
    i_vals_fg.append(img[label != 0])
i_vals_fg = np.concatenate(i_vals_fg).flatten()

_ = plt.hist(i_vals_fg, bins=100)

In [ ]:
# Stats
perc005 = np.percentile(i_vals_fg, 0.5)
perc995 = np.percentile(i_vals_fg, 99.5)
mean = np.mean(i_vals_fg)
std = np.std(i_vals_fg)

out_str = "perc005,perc995,mean,std\n"
out_str += f"{perc005},{perc995},{mean},{std}\n"
out_str += f"Used IDs (n={len(ids)})\n"
out_str += ",".join(ids)
out_str += "\n"

print(out_str)
with open(OUT_FILE, 'w') as file:
    file.write(out_str)

In [ ]:
# Before normalization
img = nib.load(os.path.join(PATH, ids[1], "registered_img.nii.gz")).get_fdata()
fig, axs = plt.subplots(2, 1)
axs[0].imshow(np.rot90(img[100, :, :], 1), cmap="gray")
_ = axs[1].hist(img.flatten(), bins=100)

In [ ]:
import sys
sys.path.append("/home/fabianb/work/vox2organ/vox2organ")
from utils.utils import global_clip_and_zscore_norm

norm_img = global_clip_and_zscore_norm(img, perc005, perc995, mean, std)

In [ ]:
# After normalization
fig, axs = plt.subplots(2, 1)
axs[0].imshow(np.rot90(norm_img[100, :, :], 1), cmap="gray")
_ = axs[1].hist(norm_img.flatten(), bins=100, range=(-2.5, 2.5))